In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn.functional as F

import sys
sys.path.append('../dgmr')
import dgmr

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [ ]:
FORECAST = 24
model = dgmr.DGMR(
    forecast_steps=FORECAST,
    input_channels=1,
    output_shape=128,
    latent_channels=384,
    context_channels=192,
    num_samples=3,
)

model = model.to(DEVICE)


In [ ]:
x = torch.rand((16, 4, 1, 128, 128)).to(DEVICE)
out = model(x)

In [ ]:
out.shape

In [ ]:
y = torch.rand((16, FORECAST, 1, 128, 128)).to(DEVICE)
loss = F.mse_loss(y, out)

In [ ]:
loss

In [ ]:
loss.shape

In [ ]:
loss.backward()